## Import Data

In [40]:
import numpy as np
import imdb 
import pandas as pd
from error_generator import Explicit_Missing_Value
from error_generator import Implicit_Missing_Value
from error_generator import White_Noise
from error_generator import Gaussian_Noise
from error_generator import Random_Active_Domain
from error_generator import Similar_Based_Active_Domain
from error_generator import Typo_Keyboard
from error_generator import Typo_Butterfingers
from error_generator import Word2vec_Nearest_Neighbor
from error_generator import Value_Selector
from error_generator import List_selected
from error_generator import Read_Write
from error_generator import Error_Generator
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

anime_data = pd.read_csv("new_anime_data1.csv")

anime_data['episodes'] = anime_data['episodes'].replace('Unknown', np.nan)


## Cleaning

In [41]:
l1 = []
l2 = []
l3 = []


anime_data['genre'].fillna('',  inplace=True)
anime_data['overview'].fillna('',  inplace=True)
anime_data['type'].fillna('',  inplace=True)
for index, row in anime_data.iterrows():
    item = row['genre']
    if(pd.isnull(item)):
            item =""
    else:
        if isinstance(item, (list, tuple)):
            item = ','.join(item)
        else:
            item = item.replace(" ","")
            item = item.replace("[","")
            item = item.replace("]","")
            item = item.replace("'","")
    l1.append(item) 
      

for index, row in anime_data.iterrows():
    item = row['overview']
    if(pd.isnull(row['overview'])):
            item = ""
    else:
       # for item in anime_data['overview']:
            if isinstance(item, (list, tuple)):
                item = ','.join(item)
            else:
                item = item.replace("[","")
                item = item.replace("]","")
    l2.append(item) 

for index, row in anime_data.iterrows():
    item = row['type']
    if(pd.isnull(row['type'])):
        item = np.nan
    else:    
        if "movie" in item:
            item = "movie"
        else:
            item = "tv series"
    l3.append(item)   
    
    
anime_data['genre'] = l1   
anime_data['overview'] = l2
anime_data['type'] = l3

#drop dublicate
anime_data.drop_duplicates(inplace = True)



## Sampling

In [42]:
anime_train, anime_test = train_test_split(anime_data, test_size=0.2)

## Generating Errors

In [43]:
def error_generater(dataset, method, n, ignored_columns):

    myselector=List_selected()

    mygen=Error_Generator()

    new_dataset=mygen.error_generator(method_gen=method,selector=myselector,percentage=n,dataset=dataset,mute_column = ignored_columns)
    
    return new_dataset

In [44]:
# generate error in the training set
new_dataset = error_generater(anime_train.values.tolist(), Random_Active_Domain(),30, ignored_columns = [0,1,5,6,7])
Read_Write.write_csv_dataset("./{}.csv".format(Random_Active_Domain().name), new_dataset)
anime_train = pd.read_csv("./{}.csv".format(Random_Active_Domain().name), 
                            names = ['Unnamed: 0', 'anime_id', 'name','genre','type', 'episodes','rating', 'members', 'overview'])

---------Change according to random_active_domain method ---------------

row: 1058 col: 2 : 'Dengeki oshioki musume Gôtaman: Gôtaman tanjôhen' changed to 'The Princess Bride'  
row: 931 col: 3 : 'Action,Adventure,Sci-Fi' changed to 'Hentai'  
row: 1495 col: 3 : 'Comedy,Horror' changed to 'Action,Comedy,Harem,Sci-Fi,Shounen,Space'  
row: 1255 col: 8 : '"It is the Taishou Era in Japan, and the Western world has begun to make its way through traditional Japan. Technology is evolving, and the use of steam has made for great breakthroughs in transportation in weaponry. But the world faces a deadly threat: a dark cult wishes to destroy Japan and the very world itself. To combat any possible threats, the Imperial government of Japan brings together six young ladies from around the world to form the Hanagumi (Flower Bridage) division of the Teikoku Kagekidan (Imperial Floral Assault Group). Clad in their proud spiritual armor, the Kobou, these young women are the frontline of defense against 

## Transformation

In [45]:
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
def get_words(x):
    bagofwords=[]
    for i in x:
        if i[1]=='NN':
            bagofwords.append(i[0])
        elif i[1]=='NNS':
            bagofwords.append(i[0])
        elif i[1]=='NNP':
            bagofwords.append(i[0])
        elif i[1]=='NNPS':
            bagofwords.append(i[0])
        elif i[1]=='JJ':
            bagofwords.append(i[0])
        elif i[1]=='JJR':
            bagofwords.append(i[0])
        elif i[1]=='JJS':
            bagofwords.append(i[0])
        elif i[1]=='RB':
            bagofwords.append(i[0])
        elif i[1]=='RBR':
            bagofwords.append(i[0])
        elif i[1]=='RBS':
            bagofwords.append(i[0])
    return bagofwords

def clean_words(x):
    b=nltk.pos_tag(nltk.word_tokenize(x))
    result=get_words(b)
    return result

In [46]:
def feature_transformation(dataset):
    #Hot_Encoding
    df = dataset.genre.str.get_dummies(',')
    
    type_lb = LabelBinarizer()
    X = type_lb.fit_transform(dataset.type.values)

    dfOneHot = pd.DataFrame(X, columns = ["movie/TVseries" for i in range(X.shape[1])])
    dataset = pd.concat([dataset, dfOneHot], axis=1)
    dataset = pd.concat([dataset, df], axis=1)

    dataset['movie/TVseries'].fillna(0, inplace=True)
    
    #Bag of Words
    summary_doc = dataset['overview'].fillna("").map(clean_words)
    summary_doc =summary_doc.apply(','.join)
 
    vectorizer = TfidfVectorizer()
    overview_feature = vectorizer.fit_transform(summary_doc).toarray()

    df = pd.DataFrame(overview_feature, columns = ["word"+ str(int(i)) for i in range(overview_feature.shape[1])])
    dataset = pd.concat([dataset, df], axis=1)
    
    dataset = dataset.drop(columns=['Unnamed: 0', 'anime_id', 'name', 'genre', 'overview', 'type'])
    
    #drop NaN values
    dataset = dataset.dropna()

    return dataset



In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
anime_train = feature_transformation(anime_train)
anime_test = feature_transformation(anime_test)

In [48]:
anime_train.shape

(1544, 6561)

In [49]:
anime_test.shape

(131, 3845)

In [50]:
anime_train_columns = list(anime_train)
anime_test_columns = list(anime_test)

for col_name in anime_train_columns:
    if col_name not in anime_test_columns:
        anime_test[str(col_name)] = 0
        
for col_name in anime_test_columns:
    if col_name not in anime_train_columns:
        anime_test.drop([str(col_name)], axis=1, inplace=True)
        

In [51]:
#anime_train = anime_train.dropna()
print(anime_train.shape)
print(anime_test.shape)

(1544, 6561)
(131, 6561)


In [52]:
anime_y_train = anime_train['rating']
anime_X_train = anime_train.drop(columns=['rating'])

anime_y_test = anime_test['rating']
anime_X_test = anime_test.drop(columns=['rating'])


## Feature Selection

In [53]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

selector = SelectKBest(score_func=f_regression,k=700)#anime_X_test.shape[1]-1)
features = selector.fit(anime_X_train, anime_y_train)

# summarize scores
np.set_printoptions(precision=3)
#print(fit.scores_)

print(anime_X_train.shape)
print(anime_X_test.shape)
anime_X_train = features.transform(anime_X_train)
anime_X_test = features.transform(anime_X_test)
print(anime_X_train.shape)
print(anime_X_test.shape)

c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


(1544, 6560)
(131, 6560)
(1544, 700)
(131, 700)


## Training - Testing - Evaluation

In [56]:
def training(model, dataset, label):
    clf = model
    clf.fit(dataset, label)
    return clf

def testing_evaluation(model, testset):
    # Make predictions using the testing set
    anime_y_pred = model.predict(testset)

    # The mean absolute error
    print("Mean absolute error: %.2f" % np.sqrt(mean_absolute_error(anime_y_test, anime_y_pred)))

    # The mean squared error
    print("Mean squared error: %.2f" % np.sqrt(mean_squared_error(anime_y_test, anime_y_pred)))

    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(anime_y_test, anime_y_pred))

### Linear Regression

In [57]:
clf = training(model = linear_model.LinearRegression(), dataset = anime_X_train, label= anime_y_train)
testing_evaluation(clf, anime_X_test)

Mean absolute error: 0.87
Mean squared error: 0.97
Variance score: -0.28


### Lesso Regression

In [58]:
clf = training(model = linear_model.Lasso(alpha=0.1), dataset = anime_X_train, label= anime_y_train)
testing_evaluation(clf, anime_X_test)

Mean absolute error: 0.77
Mean squared error: 0.77
Variance score: 0.20
